# Practical Lesson 8: Steel Company

### __Problem Description__:
A steel company has 3 plants, and each requires a minimum monthly amount of ore to operate. The company purchases ore from 3 different mines. Each mine has a maximum established monthly production capacity. Due to contractual obligations, the cost of ore for the company consists of a fixed monthly cost for each mine (this amount is paid regardless of the quantity of ore purchased from the mine), plus a transportation cost ($/t) that varies according to the distance between the mines and plants (each mine/plant pair has a different cost). The data are shown in the table below:

|            | Mine 1 ($/t) | Mine 2 ($/t) | Mine 3 ($/t) | Ore Requirement (t/month) |
|:----------:|:------------:|:------------:|:------------:|:------------------------:|
| Plant 1    | 10           | 8            | 13           | 12,300                   |
| Plant 2    | 7            | 9            | 16           | 15,400                   |
| Plant 3    | 6,5          | 10,8         | 12,6         | 13,300                   |
| **Max. mine capacity** | 11,500        | 16,500        | 13,000        | -                        |
| **Fixed cost ($)**     | 50,000        | 40,000        | 30,000        | -                        |

Build an optimization model to determine the quantity of ore to be purchased from each mine and delivered to each plant in order to minimize the total cost of ore procurement.

In [18]:
from mip import *

def solve(model):
    model.verbose = 0
    status = model.optimize()

    print("Status = ", status)
    print(f"Solution value  = {model.objective_value}\n")

    print("Solution:")
    for v in model.vars:
      if v.x > 0.00001 and v.name.find("x")!=-1:
        print(v.name, " = ", v.x)

def save(model, filename):
    model.write(filename) 
    with open(filename, "r") as f: 
        print(f.read())

In [19]:
qtd_mines = 3
qtd_plants = 3

# t_cost[i][j] : transportation cost from mine j to plant i
t_cost = [
    [10, 8, 13],
    [7, 9, 16],
    [6.5, 10.8, 12.6]
]

# f_cost[i] : fixed cost for mine i
f_cost = [
    50000,
    40000,
    30000
]

# r[i] : ore requirement for plant i
r = [
    12300,
    15400,
    13300
]

# c[i] : capacity of mine i
c = [
    11500,
    16500,
    13000
]

In [20]:
model = Model(sense=MINIMIZE, solver_name=CBC)

# x[i][j] : amount of ore transported from mine j to plant i
x = [
    [model.add_var(lb=0, var_type=CONTINUOUS, name=f"x_{i}_{j}") for j in range(qtd_mines)] for i in range(qtd_plants)
]

# y[i] : is 1 if any amount of ore is purchased from mine i
y = [model.add_var(var_type=BINARY, name=f"y_{i}") for i in range(qtd_mines)]


# Objective function
model.objective = xsum(t_cost[i][j] * x[i][j] for i in range(qtd_plants) for j in range(qtd_mines)) + \
                  xsum(f_cost[i] * y[i] for i in range(qtd_mines))

# Mines Capacity Constraints
for j in range(qtd_mines):
    model.add_constr(lin_expr=xsum(x[i][j] for i in range(qtd_plants)) <= c[j] * y[j], name=f"mine_capacity_{j}")

# Plants Demand Constraints
for i in range(qtd_plants):
    model.add_constr(lin_expr=xsum(x[i][j] for j in range(qtd_mines)) >= r[i], name=f"plant_demand_{i}")

# Show the Model
save(model, "../data/lesson8.lp")

\Problem name: 

Minimize
OBJROW: 10 x_0_0 + 8 x_0_1 + 13 x_0_2 + 7 x_1_0 + 9 x_1_1 + 16 x_1_2 + 6.50000 x_2_0 + 10.80000 x_2_1 + 12.60000 x_2_2 + 50000 y_0
 + 40000 y_1 + 30000 y_2
Subject To
mine_capacity_0:  x_0_0 + x_1_0 + x_2_0 -11500 y_0 <= -0
mine_capacity_1:  x_0_1 + x_1_1 + x_2_1 -16500 y_1 <= -0
mine_capacity_2:  x_0_2 + x_1_2 + x_2_2 -13000 y_2 <= -0
plant_demand_0:  x_0_0 + x_0_1 + x_0_2 >= 12300
plant_demand_1:  x_1_0 + x_1_1 + x_1_2 >= 15400
plant_demand_2:  x_2_0 + x_2_1 + x_2_2 >= 13300
Bounds
 0 <= y_0 <= 1
 0 <= y_1 <= 1
 0 <= y_2 <= 1
Integers
y_0 y_1 y_2 
End



In [ ]:
solve(model)